In [94]:
import pandas as pd

# загружаем данные
data = pd.read_csv('train_dataset_train.csv', sep=',')
data.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


In [95]:
columns_to_drop = [
    'id',
    'ticket_id',
    'entrance_id',
    'entrance_nm',
    'station_nm',
    'line_nm'
]
data = data.drop(columns=columns_to_drop,axis = 1)
data.head()

,ticket_type_nm,station_id,line_id,pass_dttm,time_to_under,label
0,Пропуск FacePay,11007,11,2022-09-12 05:00:13,216.316667,8001
1,СК учащегося 30 дней,2006,2,2022-09-12 05:00:54,648.183333,9011
2,БСК дружинника г.Москвы,2006,2,2022-09-12 05:00:55,865.333333,7022
3,30 дней,2006,2,2022-09-12 05:01:13,1048.233333,2022
4,КОШЕЛЕК,2006,2,2022-09-12 05:02:55,965.600000,2017


In [96]:
data['pass_dttm'] = pd.to_datetime(data['pass_dttm'])
data['hour'] = data['pass_dttm'].dt.hour

In [97]:
columns_to_drop = ['pass_dttm']
data = data.drop(columns=columns_to_drop,axis = 1)

In [98]:
data.head()

,ticket_type_nm,station_id,line_id,time_to_under,label,hour
0,Пропуск FacePay,11007,11,216.316667,8001,5
1,СК учащегося 30 дней,2006,2,648.183333,9011,5
2,БСК дружинника г.Москвы,2006,2,865.333333,7022,5
3,30 дней,2006,2,1048.233333,2022,5
4,КОШЕЛЕК,2006,2,965.600000,2017,5


In [99]:
arr = [i+1 for i in range(61)]
n = list(data['ticket_type_nm'].unique())
data.ticket_type_nm.replace({n[i]:arr[i] for i in range(len(n))},inplace=True)

In [100]:
data.head()

,ticket_type_nm,station_id,line_id,time_to_under,label,hour
0,1,11007,11,216.316667,8001,5
1,2,2006,2,648.183333,9011,5
2,3,2006,2,865.333333,7022,5
3,4,2006,2,1048.233333,2022,5
4,5,2006,2,965.600000,2017,5


In [101]:
df_pred = data['time_to_under']
data.drop(['time_to_under'],axis=1,inplace=True)

In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, df_pred, test_size=0.3)

In [103]:
### предсказываем time_to_under

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test,y_pred)
r2

0.4607733461420076

In [105]:
df_pred = data['label']
data2 = data.drop(['label'],axis=1)

In [106]:
## предсказываем lable

In [107]:
from sklearn.tree import DecisionTreeClassifier
logreg = DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(data2, df_pred, test_size=0.1)
#X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.3)
logreg.fit(X_train, y_train)
y_predict = logreg.predict(X_test)

In [108]:
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_predict, average='micro')
print(recall)

0.10375516713564246


In [109]:
res = 0.5*r2+0.5*recall
res

0.28226425663882504